In [ ]:
# Importeer benodigde modules
import os
import shutil
import numpy as np
from PIL import Image 
import ipywidgets as widgets
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor

# Label Assembly
Dit script is bedoeld om gelabelde segmenten samen te plakken tot de originele afbeelding met een bijbehorend label. Het script kan verschillende afbeeldingen en afbeelding formaten verwerken en berekend de correcte afmetingen per afbeelding. Wanneer er verschillende Label.txt bestanden gevonden worden met unieke inhoud word er van iedere unieke inhoud een versie opgeslagen met een oplopend nummer in de naam. Een Label word pas mee genomen in de output als er zowel een label als een bijbehorend segment aangeleverd word. Voor de segmenten zelf kan deze overweging door de gebruiker ingesteld worden. 

**LET OP!** Bij het draaien van het script wordt de opgegeven OutputLocatie opgeruimd, waarbij alle bestanden in deze map verwijderd wordt.

---
<blockquote style="border-left: 5px solid red; padding-left: 10px;">

## **Attentie!** 
<code> Update onderstaande cell voordat je de het script runt </code></blockquote>
 

In [ ]:
# Input map met de segmenten en labels.
InputAnnotaties : list[str] = [r"pad/naar/jouw/folder", r"pad/naar/jouw/andere/folder"]
# Bijvoorbeeld: [r"pad/naar/jouw/folder", r"pad/naar/jouw/andere/folder"]

# Output map waar de "Stitched Images en labels" worden opgeslagen.
OutputLocatie : str = r"Nieuwe Map"
# Bijvoorbeeld: r"pad/naar/jouw/output/folder" OF r"Nieuwe Map"

# Bepaal of de segmenten(afbeeldingen) zonder labelbestanden terug komen in de het stitch geheel.
KeepOnlyLabeled : bool = True
# Bijvoorbeeld: True OF False

---
## **Code**

In [ ]:
#_Collect base info______________________________________________________________________________________________________________________
# Controleren of de outputfolder bestaat en leeg is en maak alle benodigde outputlocaties aan
if os.path.exists(OutputLocatie):
    shutil.rmtree(OutputLocatie)

OutputFolderStructure = {
    "Stitched Images" : os.path.join(OutputLocatie, "Stitched", "images"),
    "Stitched Labels" : os.path.join(OutputLocatie, "Stitched", "labels"),
    "Segmented Images" : os.path.join(OutputLocatie, "Segmented", "images"),	
    "Segmented Labels" : os.path.join(OutputLocatie, "Segmented", "labels")
}

for OutputMappen in OutputFolderStructure.values():
    os.makedirs(OutputMappen)

#ondersteunde bestandstypen
ImageTypes = [".jpg", ".png", "jpeg"]
ImageTypes = tuple(Type.lower() for Type in ImageTypes)
LabelTypes = [".txt"]
LabelTypes = tuple(Type.lower() for Type in LabelTypes)

# Bekijkt alle bestanden in InputAnnotaties die "Labels.txt" heten en verzameld alle unieke variaties
Labels_txt = {
    "Label Text":[],
    "Unieke Labels":[]
}
for Folders in InputAnnotaties:
    for Root, Dirs, Files in os.walk(Folders):
        for File in Files:
            if File == "Labels.txt":
                with open(os.path.join(Folders, Root, File), "r") as Text:
                    if Text.read() not in Labels_txt["Label Text"]:
                        Labels_txt["Label Text"].append(Text.read())
                        Labels_txt["Unieke Labels"].append(os.path.join(os.path.join(Folders, Root, File)))

# Als er meer dan een unieke "Labels.txt" Bestaat worden ze genummerd gekopieerd anders behoud het de originele naam
if len(Labels_txt["Unieke Labels"]) > 1:
    for i in range(0, len(Labels_txt["Unieke Labels"])):
        shutil.copy2(Labels_txt["Unieke Labels"][i], os.path.join(OutputFolderStructure["Stitched Labels"], f"Labels({i+1}).txt"))
        shutil.copy2(Labels_txt["Unieke Labels"][i], os.path.join(OutputFolderStructure["Segmented Labels"], f"Labels({i+1}).txt"))
elif len(Labels_txt["Unieke Labels"]) == 1:
    shutil.copy2(Labels_txt["Unieke Labels"][0], os.path.join(OutputFolderStructure["Stitched Labels"], f"Labels.txt"))

# Verzameld alle Labels en afbeeldingen in de InputAnnotaties per originele afbeelding en extraheert de coördinaten uit de naam zodat de grote van de originele afbeeldingen berekend kan worden
BaseImages = {}
for Folders in InputAnnotaties:
    for Root, Dirs, Files in os.walk(Folders):
        for File in Files:
            FileStem, FileType = os.path.splitext(File)
            if not File == "Labels.txt":
                BaseName, YCoördinaat, XCoördinaat = FileStem.rsplit("_", 2)

                if BaseName not in BaseImages.keys():
                    BaseImages[BaseName] = {
                        "Images" : {},
                        "Labels" : {},
                        "YCoördinaten" : set(),
                        "XCoördinaten" : set()
                    }

                if FileType.lower() in ImageTypes:
                    BaseImages[BaseName]["Images"][FileStem] = os.path.join(Folders, Root, File)
                    BaseImages[BaseName]["YCoördinaten"].add(int(YCoördinaat))
                    BaseImages[BaseName]["XCoördinaten"].add(int(XCoördinaat))

                elif FileType.lower() in LabelTypes:
                    BaseImages[BaseName]["Labels"][FileStem] = os.path.join(Folders, Root, File)
                    BaseImages[BaseName]["YCoördinaten"].add(int(YCoördinaat))
                    BaseImages[BaseName]["XCoördinaten"].add(int(XCoördinaat))


#_User Interface______________________________________________________________________________________________________________________
# Functie voor het maken van een knop. Functie stroomlijnt de aanmaak en centreert de instellingen voor gemakkelijke aanpassingen
def MakeButton(description, icon):
    Button = widgets.Button(
        description=description, 
        icon=icon, 
        style={'text_align': 'center'}
        )
    Button.style.button_color = 'aquamarine'
    Button.layout = widgets.Layout(display='flex', justify_content='flex-start', align_items='center')
    return Button

# Maakt outputschermen voor de UI
OutputScreen = widgets.Output()
OutputLoadingScreen = widgets.Output()
OutputTextScreen = widgets.Output()


Kaartjes = ["*empty*"] # Lijst met de outputs
Index = 0 # Houd de huidige weergave bij
BigStep = 5 # bepaalt hoe groot de stap vooruit en achteruit is op de UI

# Maakt de Output weergave
TextCard = widgets.Textarea(
    value=Kaartjes[Index], 
    disabled=True, 
    layout=widgets.Layout(
        width='90%', 
        height='200px'
    ))


# Maakt de knopen om de Output weergave te navigeren
ButtonFullBack = MakeButton("Eerste", "backward")
ButtonBigBack = MakeButton(f"Ga {BigStep} terug", "fast-backward")
ButtonBack = MakeButton("Ga 1 terug", "step-backward")
ButtonForward = MakeButton("Ga 1 vooruit", "step-forward")
ButtonBigForward = MakeButton(f"Ga {BigStep} vooruit", "fast-forward")
ButtonFullForward = MakeButton("Laatste", "forward")

OutputButtons = [
    ButtonFullBack,
    ButtonBigBack,
    ButtonBack,
    ButtonForward,
    ButtonBigForward,
    ButtonFullForward
]

# los aangemaakt zodat de waarden automaties updaten wanneer ik hem aanroep
ProgressBarDescription = lambda : f"Je bekijkt output {Index + 1} van {len(Kaartjes)}:" 

# Weergeeft welke output ik bekijk en hoeveel er in totaal zijn
ProgressBar = widgets.IntProgress(
        value=Index+1,
        max=len(Kaartjes),
        description= ProgressBarDescription(),
        style={"description_width": "initial"},
        layout=widgets.Layout(width="90%"),
    )

# Functie om de UI weergave te updaten
def UpdateProgressBar():
    ProgressBar.max=len(Kaartjes)
    ProgressBar.value = Index+1
    ProgressBar.description = ProgressBarDescription()

    TextCard.value = Kaartjes[Index]

# Functie om "Kaartjes" en "Index" te updaten wanneer een afbeelding af is.
# Moet los van UpdateTextKaart() omdat anders het script staakt na de eerste afbeelding die gesticht is
def AddTextCard(Text):
    global Index
    Kaartjes.append(Text)
    if "*empty*" in Kaartjes:
         Kaartjes.pop(Kaartjes.index("*empty*"))
    elif Index == len(Kaartjes) - 2:	
        Index = len(Kaartjes) - 1
    UpdateProgressBar()

# # Functie om "Index" en daarmee de UI te Updaten als gevolg van User input.
def UpdateTextKaart(change):
    global Index
    if change.description == ButtonFullBack.description:
        Index = 0
    elif change.description == ButtonBigBack.description and Index >= BigStep:
        Index -= 5
    elif change.description == ButtonBack.description and Index > 0:
        Index -= 1
    elif change.description == ButtonForward.description and Index < len(Kaartjes) - 1:
        Index += 1
    elif change.description == ButtonBigForward.description and Index < len(Kaartjes) - BigStep:
        Index += 5
    elif change.description == ButtonFullForward.description:
        Index = len(Kaartjes)-1
    UpdateProgressBar()

# Zorcht dat het drukken op knoppen geregistreerd word
for Button in OutputButtons:
    Button.on_click(UpdateTextKaart)

# Display de widgets
DisplayTextOutput = widgets.HBox([
                        widgets.VBox([
                            TextCard,
                            ProgressBar
                        ], layout=widgets.Layout(width='80%', align_items='center')),
                        widgets.VBox(OutputButtons)
                    ], layout=widgets.Layout(width='1000px', height='270px', align_items='center'))

with OutputScreen:
    display(OutputLoadingScreen, OutputTextScreen)

with OutputTextScreen:
    display(DisplayTextOutput)

#_Stich nieuwe afbeeldingen en labels______________________________________________________________________________________________________________________
# deze loop is een functie zodat hij op een losse tread gedraaid kan worden
# Doe je dit niet is de UI niet bruikbaar zolang het script nog bezig is een kun je pas door de outputs heen kijken wanneer alle afbeeldingen gesticht zijn
def StitchingFunction():
    global Kaartjes, BaseImages, OutputFolderStructure, OutputLoadingScreen
    with OutputLoadingScreen:
        # loop om een voor een de afbeeldingen en de labels samen te stitchen
        for BaseFile in tqdm(BaseImages.keys(), desc="Afbeeldingen en labels samenvoegen", leave=True):

            # Maakt lijst van alle segmenten waar ook labels van zijn
            KeysCompleteAnnotaties = []
            for Key in BaseImages[BaseFile]["Images"].keys():
                if Key in BaseImages[BaseFile]["Labels"].keys():
                    KeysCompleteAnnotaties.append(Key)

            # Checkt of er label en segment combinaties zijn om te stichen
            if not len(KeysCompleteAnnotaties) > 0:
                AddTextCard(f"Geen match gevonden van labels met segmenten voor {BaseFile}")
            else:
                # bepaal de grote van de segmenten en zet de spreiding op een maximum van een segment
                # Aanname: de segmenten zijn vierkant
                # Aanname: de segmenten van één afbeelding zijn allemaal even groot
                SegmentGroote = 0
                SegmentSpreiding = 0

                with Image.open(BaseImages[BaseFile]["Images"][KeysCompleteAnnotaties[0]]) as ImageFile:
                    SegmentGroote, SegmentSpreiding = ImageFile.size

                # Bepaal of er overlap is tussen de segmenten. 
                # Aanname: de segmenten zijn vierkant
                # Dus kunnen x en y coördinaten samen vergeleken worden
                # Dit zorgt voor een stabieler script wanneer er een fractie van de segmenten zijn aangeleverd
                ListXCoördinaten = list(BaseImages[BaseFile]["XCoördinaten"])
                ListYCoördinaten = list(BaseImages[BaseFile]["YCoördinaten"])
                listAllCoördinaten = ListXCoördinaten + ListYCoördinaten

                for FirstValue in listAllCoördinaten:
                    for SecondValue in listAllCoördinaten:
                        if SegmentSpreiding > abs(SecondValue - FirstValue) and abs(SecondValue - FirstValue) != 0:
                            SegmentSpreiding = abs(SecondValue - FirstValue)
                
                # Berekend de afmetingen van de originele afbeelding op basis van het grootste x en y coördinaat bij die foto.
                # Ieder coördinaat is van de linker bovenhoek van het segment, om hiervoor te compenseren word er een SegmentGroote aan pixels aan de Grootste x en y waarde toegevoegd
                ListXCoördinaten.sort()
                ListYCoördinaten.sort()

                BaseWidth = ListXCoördinaten[-1] + SegmentGroote
                BaseHeight = ListYCoördinaten[-1] + SegmentGroote
                BaseWidthCount = BaseWidth // SegmentSpreiding
                BaseHeightCount = BaseHeight // SegmentSpreiding

                # maakt een lege tabel ter grote van de nieuwe afbeelding met 3 lagen (RGB) en 8 bits per cel (Genoeg voor een kleurwaarde)
                StitchedImage = np.zeros((BaseHeight, BaseWidth, 3), dtype=np.uint8)

                # loopt over de geselecteerde selectie segmenten heen en leest ze in tabel met kleurwaarde.
                # De nieuwe tabel word op basis van de coördinaten in de naam van de foto in de StitchedImage tabel geplakt
                KeysToUse = KeysCompleteAnnotaties if KeepOnlyLabeled else BaseImages[BaseFile]["Images"].keys()
                for Key in KeysToUse:
                    with Image.open(BaseImages[BaseFile]["Images"][Key]) as ImageFile:
                        ImageNpArray = np.array(ImageFile)
                        _, YCoördinaat, XCoördinaat = Key.rsplit("_", 2)


                        StitchedImage[
                            int(YCoördinaat) : (int(YCoördinaat) + SegmentGroote), 
                            int(XCoördinaat) : (int(XCoördinaat) + SegmentGroote)
                            ] = ImageNpArray

                        # slaat het losse segment op zodat duidelijk is welke segmenten wel en niet mee genomen zijn
                        SaveLocationFile = BaseImages[BaseFile]["Images"][Key].rsplit("\\", 1)[1]
                        ImageFile.save(os.path.join(OutputFolderStructure["Segmented Images"], SaveLocationFile))
                
                # Zet de StitchedImage om naar een jpg bestand en slaat hem op in de output
                StitchedImage = Image.fromarray(StitchedImage)
                SaveLocationStitchedImage = os.path.join(OutputFolderStructure["Stitched Images"], f"{BaseFile}.jpg")
                StitchedImage.save(SaveLocationStitchedImage)
                StitchedImage.close()

                # Zorgt dat het nieuwe label bestand bestaat en leeg is
                StitchedLabelName = f"{BaseFile}.txt"
                SaveLocationStitchedLabel = os.path.join(OutputFolderStructure["Stitched Labels"], StitchedLabelName) 
                with open(SaveLocationStitchedLabel, "w+") as LabelFile:
                    LabelFile.write("")
                
                # Loopt over alle geselecteerde labels heen en berekend de nieuwe locatie in de afbeelding.
                with open(SaveLocationStitchedLabel, "+a") as LabelFile:
                    for Key in KeysCompleteAnnotaties:
                        _, YCoördinaat, XCoördinaat = Key.rsplit("_", 2)
                        SaveLocationOgFile = BaseImages[BaseFile]["Labels"][Key]
                        SaveLocationFile = os.path.join(OutputFolderStructure["Segmented Labels"], SaveLocationOgFile.rsplit("\\", 1)[1])
                        shutil.copy2(SaveLocationOgFile, SaveLocationFile) # slaat het losse label op zodat duidelijk is welke lebels wel en niet mee genomen zijn

                        with open(BaseImages[BaseFile]["Labels"][Key], "r") as LabelSegment:
                            Annotaties = LabelSegment.readlines()
                            for Bbox in Annotaties:
                                BboxSplit = [float(Element) for Element in Bbox.split(" ")]

                                BboxLabel = int(BboxSplit[0])
                                BboxX = (BboxSplit[1] * SegmentGroote + int(XCoördinaat))/BaseWidth
                                BboxY = (BboxSplit[2] * SegmentGroote + int(YCoördinaat))/BaseHeight
                                BboxWidth = (BboxSplit[3] * SegmentGroote)/BaseWidth
                                BboxHeight = (BboxSplit[4] * SegmentGroote)/BaseHeight

                                LabelFile.write(f"{BboxLabel} {BboxX} {BboxY} {BboxWidth} {BboxHeight}\n")
                
                # Maakt een samenvattende output Voor de gebruiker om inzicht te krijgen in de output afbeeldingen en labels
                AddTextCard(f"""
FileName: {BaseFile}
segment Groote: {SegmentGroote} pixels
Overlap tussen segmenten: {(SegmentGroote-SegmentSpreiding)/SegmentGroote*100}%
Afmetingen stitched image:  {BaseWidth}x{BaseHeight} pixels of {BaseWidthCount}x{BaseHeightCount} segmenten
Gevonden Complete segmenten: {len(KeysCompleteAnnotaties)} van de {BaseWidthCount*BaseHeightCount} segmenten

SaveLocationStitchedImage: {SaveLocationStitchedImage}
SaveLocationStitchedLabel: {SaveLocationStitchedLabel}
""")

# Maakt nieuwe treads aan om de afbeeldingen te maken zonder het gebruik van de UI te stoppen
executor = ThreadPoolExecutor(max_workers=1)
future = executor.submit(StitchingFunction)
